# FM

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np
from sklearn.datasets import make_circles

# 加载同心圆数据
xs, ys = make_circles(600, noise=0.1, factor=0.2)
ys = ys * 2 - 1  # 0/1转换为-1/1

# 特征维数
FEAT_DIM = 20
# 构造噪声特征
xs = np.concatenate([xs, np.random.normal(0.0, 0.01, (600, FEAT_DIM - 2))], axis=1)
# 隐藏向量维度
HIDDEN_DIM = 2

print(xs.shape, ys.shape)
print(xs[0], ys[0])

(600, 20) (600,)
[ 1.43629708e-01 -1.91212198e-01  2.60332837e-04  1.90875232e-03
 -8.54440868e-03 -4.50469881e-03 -6.22663329e-03 -1.40363399e-02
  3.26462414e-03  1.82013032e-02  4.44894307e-03  1.32349584e-02
  1.25476752e-02  7.46143871e-03 -6.80912229e-03 -1.77366370e-04
 -1.38554174e-02  1.09058037e-02  4.84247244e-03 -4.56575055e-03] 1


In [3]:
# 训练
from mflow import core, ops, opts

# 超参数
lr = 0.001
epoch = 50
batch_size = 16

with core.NameScope("FM"):
    # 初始化变量
    x = core.Variable(size=(FEAT_DIM, 1), trainable=False)
    y = core.Variable(size=(1, 1), trainable=False)
    w = core.Variable(size=(1, FEAT_DIM), trainable=True)
    h = core.Variable(size=(HIDDEN_DIM, FEAT_DIM), trainable=True)
    b = core.Variable(size=(1, 1), trainable=True)
    # 模型定义
    pred = ops.Add(
        ops.MatMal(
            ops.Reshape(x, shape=(1, FEAT_DIM)), 
            ops.MatMal(ops.MatMal(ops.Reshape(h, shape=(FEAT_DIM, HIDDEN_DIM)), h), x)
        ),  # 二次项
        ops.MatMal(w, x),  # 一次项
        b  # 偏置项
    )
    predicter = ops.Logistic(pred)
    loss = ops.loss.LogLoss(ops.Multiply(y, pred))
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, lab) in enumerate(zip(xs, ys)):
            x.setValue(np.mat(feat).T)
            y.setValue(np.mat(lab))
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in xs:
            x.setValue(np.mat(feat).T)
            predicter.forward()
            preds.append(predicter.value[0, 0])  # 结果
        preds = (np.array(preds) > 0.5).astype("int") * 2 - 1
        acc = (ys == preds).astype("uint8").sum() / len(xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.672.
Epoch: 2, acc: 0.940.
Epoch: 3, acc: 0.943.
Epoch: 4, acc: 0.918.
Epoch: 5, acc: 0.908.
Epoch: 6, acc: 0.910.
Epoch: 7, acc: 0.925.
Epoch: 8, acc: 0.932.
Epoch: 9, acc: 0.943.
Epoch: 10, acc: 0.955.
Epoch: 11, acc: 0.960.
Epoch: 12, acc: 0.965.
Epoch: 13, acc: 0.970.
Epoch: 14, acc: 0.972.
Epoch: 15, acc: 0.975.
Epoch: 16, acc: 0.978.
Epoch: 17, acc: 0.985.
Epoch: 18, acc: 0.988.
Epoch: 19, acc: 0.990.
Epoch: 20, acc: 0.988.
Epoch: 21, acc: 0.988.
Epoch: 22, acc: 0.988.
Epoch: 23, acc: 0.988.
Epoch: 24, acc: 0.990.
Epoch: 25, acc: 0.990.
Epoch: 26, acc: 0.990.
Epoch: 27, acc: 0.990.
Epoch: 28, acc: 0.990.
Epoch: 29, acc: 0.990.
Epoch: 30, acc: 0.992.
Epoch: 31, acc: 0.992.
Epoch: 32, acc: 0.992.
Epoch: 33, acc: 0.993.
Epoch: 34, acc: 0.993.
Epoch: 35, acc: 0.995.
Epoch: 36, acc: 0.997.
Epoch: 37, acc: 0.998.
Epoch: 38, acc: 1.000.
Epoch: 39, acc: 1.000.
Epoch: 40, acc: 1.000.
Epoch: 41, acc: 1.000.
Epoch: 42, acc: 1.000.
Epoch: 43, acc: 1.000.
Epoch: 44, acc: 1.00